<a href="https://colab.research.google.com/github/GeenDutchman/cs474_labs/blob/master/DL_Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a 
href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab6.ipynb"
  target="_parent">
  <img
    src="https://colab.research.google.com/assets/colab-badge.svg"
    alt="Open In Colab"/>
</a>

# Lab 6: Sequence-to-sequence models

### Description:
For this lab, you will code up the [char-rnn model of Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). This is a recurrent neural network that is trained probabilistically on sequences of characters, and that can then be used to sample new sequences that are like the original.

This lab will help you develop several new skills, as well as understand some best practices needed for building large models. In addition, we'll be able to create networks that generate neat text!

### Deliverable:
- Fill in the code for the RNN (using PyTorch's built-in GRU).
- Fill in the training loop
- Fill in the evaluation loop. In this loop, rather than using a validation set, you will sample text from the RNN.
- Implement your own GRU cell.
- Train your RNN on a new domain of text (Star Wars, political speeches, etc. - have fun!)

### Grading Standards:
- 20% Implementation the RNN
- 20% Implementation training loop
- 20% Implementation of evaluation loop
- 20% Implementation of your own GRU cell
- 20% Training of your RNN on a domain of your choice

### Tips:
- Read through all the helper functions, run them, and make sure you understand what they are doing
- At each stage, ask yourself: What should the dimensions of this tensor be? Should its data type be float or int? (int is called `long` in PyTorch)
- Don't apply a softmax inside the RNN if you are using an nn.CrossEntropyLoss (this module already applies a softmax to its input).

### Example Output:
An example of my final samples are shown below (more detail in the
final section of this writeup), after 150 passes through the data.
Please generate about 15 samples for each dataset.

<code>
And ifte thin forgision forward thene over up to a fear not your
And freitions, which is great God. Behold these are the loss sub
And ache with the Lord hath bloes, which was done to the holy Gr
And appeicis arm vinimonahites strong in name, to doth piseling 
And miniquithers these words, he commanded order not; neither sa
And min for many would happine even to the earth, to said unto m
And mie first be traditions? Behold, you, because it was a sound
And from tike ended the Lamanites had administered, and I say bi
</code>


---

## Part 0: Readings, data loading, and high level training

---

There is a tutorial here that will help build out scaffolding code, and get an understanding of using sequences in pytorch.

* Read the following

> * [Pytorch sequence-to-sequence tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) (Take note that you will not be implementing the encoder part of this tutorial.)
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)






In [124]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

import unidecode
import string
import random
import re
 
import pdb
 
all_characters = string.printable
n_characters = len(all_characters)
file = unidecode.unidecode(open('./text_files/lotr.txt').read())
file_len = len(file)
print('file_len =', file_len)

--2020-10-09 04:08:28--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 52.5.213.57, 3.224.25.41, 54.205.50.50, ...
Connecting to piazza.com (piazza.com)|52.5.213.57|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2020-10-09 04:08:29--  https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving d1b10bmlvqabco.cloudfront.net (d1b10bmlvqabco.cloudfront.net)... 54.230.85.211, 54.230.85.196, 54.230.85.168, ...
Connecting to d1b10bmlvqabco.cloudfront.net (d1b10bmlvqabco.cloudfront.net)|54.230.85.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[====

In [125]:
chunk_len = 200
 
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]
  
print(random_chunk())

 all the old names.' 

'It is also called kingsfoil,' said Aragorn; 'and maybe you know it by 
that name, for so the country-folk call it in these latter days.' 'Oh that!' 
said Ioreth. 'Well, if your 


In [126]:
import torch
# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
      tensor[c] = all_characters.index(string[c])
  return tensor

abcDEF_tensor = char_tensor('abcDEF')
print(abcDEF_tensor)

def tensor_char(tensor):
  result_str = ''
  for l in range(tensor.size()[0]):
    result_str += all_characters[tensor[l]]
  return result_str

abcDEF_string = tensor_char(abcDEF_tensor)
print(abcDEF_string)
# twice_tensor = abcDEF_tensor.append(abcDEF_tensor)
# print(tensor_char(twice_tensor))

tensor([10, 11, 12, 39, 40, 41])
abcDEF


---

## Part 4: Creating your own GRU cell 

**(Come back to this later - its defined here so that the GRU will be defined before it is used)**

---

The cell that you used in Part 1 was a pre-defined Pytorch layer. Now, write your own GRU class using the same parameters as the built-in Pytorch class does.

Please try not to look at the GRU cell definition. The answer is right there in the code, and in theory, you could just cut-and-paste it. This bit is on your honor!

**TODO:**
* Create a custom GRU cell

**DONE:**



In [127]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(GRU, self).__init__()
    
  
  def forward(self, inputs, hidden):
    # Each layer does the following:
    # r_t = sigmoid(W_ir*x_t + b_ir + W_hr*h_(t-1) + b_hr)
    # z_t = sigmoid(W_iz*x_t + b_iz + W_hz*h_(t-1) + b_hz)
    # n_t = tanh(W_in*x_t + b_in + r_t**(W_hn*h_(t-1) + b_hn))
    # h_(t) = (1 - z_t)**n_t + z_t**h_(t-1)
    # Where ** is hadamard product (not matrix multiplication, but elementwise multiplication)
    
    return outputs, hiddens
  


---

##  Part 1: Building a sequence to sequence model

---

Great! We have the data in a useable form. We can switch out which text file we are reading from, and trying to simulate.

We now want to build out an RNN model, in this section, we will use all built in Pytorch pieces when building our RNN class.


**TODO:**

**DONE:**
* Create an RNN class that extends from nn.Module.




In [128]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1, soft_max=False):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers
    
    # more stuff here...
    self.embedding = nn.Embedding(self.output_size, self.hidden_size)
    self.gru = nn.GRU(input_size=self.hidden_size, hidden_size=self.hidden_size, num_layers=self.n_layers)
    self.linOut = nn.Linear(self.hidden_size, self.output_size)
    self.softmax = nn.Softmax(dim=1) if soft_max is True else None

  def forward(self, input_char, hidden):
    # by reviewing the documentation, construct a forward function that properly uses the output
    # of the GRU
    # https://pytorch.org/docs/stable/generated/torch.nn.GRU.html

    # stuff here
    out_decoded = self.embedding(input_char).view(1, 1, -1)
    out_decoded = F.relu(out_decoded)
    out_decoded, hidden = self.gru(out_decoded, hidden)
    out_decoded = self.linOut(out_decoded[0])
    if self.softmax is not None:
      out_decoded = self.softmax(out_decoded)
    
    return out_decoded, hidden

  def init_hidden(self):
    return torch.zeros(self.n_layers, 1, self.hidden_size)

In [129]:
def random_training_set():    
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

print(random_training_set())

(tensor([28, 94, 21, 18, 20, 14, 94, 27, 10, 18, 23, 94, 24, 23, 94, 10, 94, 32,
        18, 21, 29, 14, 13, 94, 21, 14, 29, 29, 30, 12, 14, 75, 94, 55, 17, 14,
        27, 14, 62, 94, 49, 24, 32, 94, 44, 94, 12, 10, 23, 94, 20, 14, 14, 25,
        94, 96, 10, 32, 10, 20, 14, 94, 21, 24, 23, 16, 94, 14, 23, 24, 30, 16,
        17, 94, 29, 24, 94, 14, 10, 29, 94, 10, 94, 11, 18, 29, 75, 68, 94, 96,
        96, 55, 17, 14, 34, 94, 32, 14, 27, 14, 94, 21, 14, 13, 94, 29, 17, 14,
        23, 94, 29, 24, 94, 28, 14, 10, 29, 28, 94, 11, 14, 28, 18, 13, 14, 94,
        41, 10, 27, 10, 22, 18, 27, 77, 94, 11, 10, 27, 27, 14, 21, 28, 94, 12,
        24, 31, 14, 27, 14, 13, 94, 32, 18, 29, 17, 94, 25, 14, 21, 29, 28, 94,
        96, 10, 23, 13, 94, 17, 18, 16, 17, 94, 14, 23, 24, 30, 16, 17, 94, 10,
        11, 24, 31, 14, 94, 29, 17, 14, 94, 11, 14, 23, 12, 17, 14, 28, 94, 24,
        15, 94]), tensor([94, 21, 18, 20, 14, 94, 27, 10, 18, 23, 94, 24, 23, 94, 10, 94, 32, 18,
        21, 29, 14, 1

---

## Part 2: Sample text and Training information

---

We now want to be able to train our network, and sample text after training.

This function outlines how training a sequence style network goes. 

**TODO:**

**DONE:**
* Fill in the pieces.





In [130]:
# NOTE: decoder_optimizer, decoder, and criterion will be defined below as global variables
def train(inp, target, per_char=True, teacher_forcing=True):
  ## initialize hidden layers, set up gradient and loss 
    # your code here
  ## /
  inp_len = inp.size(0)
  target_len = target.size(0)

  decoder.train()
  decoder_optimizer.zero_grad()
  hidden = decoder.init_hidden()
  loss = 0
  
  # result = []
  # more stuff here...
  if per_char:
    next_char = inp[0]
    for index in range(inp_len):
      output, hidden = decoder(next_char, hidden)
      values, indicies = output.topk(1)
      # print(values, indicies)
      result_char = indicies.squeeze().detach()
      # result.append(result_char)
      if teacher_forcing:
        next_char = target[index]
      else:
        next_char = result_char
      # output.transpose_(1, 2)
      # print(output.size(), output.squeeze().size(), target[index].view(-1).size())
      loss += criterion(output, target[index].view(-1))

  else:
    result, hidden = decoder(inp, hidden)
    loss += criterion(result, target)
  
  loss.backward()
  decoder_optimizer.step()
  
  return loss.item() / target_len


---

## Part 3: Sample text and Training information

---

You can at this time, if you choose, also write out your train loop boilerplate that samples random sequences and trains your RNN. This will be helpful to have working before writing your own GRU class.

If you are finished training, or during training, and you want to sample from the network you may consider using the following function. If your RNN model is instantiated as `decoder`then this will probabilistically sample a sequence of length `predict_len`

**TODO:**

**DONE:**
* Fill out the evaluate function to generate text frome a primed string




In [131]:
def sample_outputs(output, temperature):
    """Takes in a vector of unnormalized probability weights and samples a character from the distribution"""
    # print(output.size())
    return torch.multinomial(torch.exp(output / temperature), 1)

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  ## initialize hidden state, initialize other useful variables
    # your code here
  ## /
  torch.no_grad()
  decoder.eval()
  result_str = ''
  tensor_str = char_tensor(prime_str)

  hidden = decoder.init_hidden()

  predicted = []

  next_char = 0 # declare it out of loop scope
  for i in range(predict_len):
    next_char = tensor_str[i] if i < tensor_str.size()[0] else next_char
    # print(next_char, next_char < len(all_characters), len(all_characters))
    output, hidden = decoder(next_char, hidden)
    next_char = sample_outputs(output, temperature)
    predicted.append(next_char)

  predicted = torch.cat(predicted)
  result_str = tensor_char(predicted)

  return result_str


# def scope(epoch):
#   gc.collect()  

---

## Part 4: (Create a GRU cell, requirements above)

---



---

## Part 5: Run it and generate some text!

---

Assuming everything has gone well, you should be able to run the main function in the scaffold code, using either your custom GRU cell or the built in layer, and see output something like this. I trained on the “lotr.txt” dataset, using chunk_length=200, hidden_size=100 for 2000 epochs gave.

**TODO:** 
* Create some cool output

**DONE:**



In [132]:
import time
n_epochs = 5000
print_every = 200
plot_every = 10
hidden_size = 200
n_layers = 3
lr = 0.001
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0

In [ ]:
# n_epochs = 2000 
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

[123.56751537322998 (200 4%) 2.3801]
W Gga conden wed an wexr Tlilny Bir Sot lai. 
mes bas, aus somgels bhe nleun. 
'
bent mot carod wmpo 

[248.59755778312683 (400 8%) 2.1676]
oe has to cut ine apor fare Eumink . cars ,cong of rorth am sad in store the nor hoth no 
the hirt i 

[372.1814432144165 (600 12%) 1.6855]
einet Jore and a she come rievs ut clood though and so the mack was of they ladd 
in the your lode w 

[494.46493911743164 (800 16%) 1.5524]
eeven was now the 
cpace manes all the asher.' 

'But lite am? 

'Waessed and beting was riepling,'  

[618.414496421814 (1000 20%) 1.8164]
han a can dam. It man wair have to their whed and Porn of with the grid, and the row, and at that mi 

[743.0812449455261 (1200 24%) 1.6310]
houved, and 
foing to 
esanding was licking there wat little 
to some canted they can 













 

[866.067773103714 (1400 28%) 1.8820]
hing his neage. He could dound it urned. 

'There cook light him, and tree to the find kangern quick 

[992.3939967155457

In [ ]:
for i in range(10):
  start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
  start = random.randint(0,len(start_strings)-1)
  print(start_strings[start])
#   all_characters.index(string[c])
  print(evaluate(start_strings[start], 200), '\n')

---

## Part 6: Generate output on a different dataset

---

**TODO:**

* Choose a textual dataset. Here are some [text datasets](https://www.kaggle.com/datasets?tags=14104-text+data%2C13205-text+mining) from Kaggle 

* Generate some decent looking results and evaluate your model's performance (say what it did well / not so well)

**DONE:**

